In [1]:
!git clone https://github.com/brjathu/SKD.git

Cloning into 'SKD'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 96 (delta 38), reused 49 (delta 9), pack-reused 0
Unpacking objects: 100% (96/96), 2.30 MiB | 3.71 MiB/s, done.


In [2]:
import wandb
wandb_api = "xxxxxxxxxxxxxxxx"

wandb.login(key=wandb_api)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
pip install wandb --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.2 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: wandb
    Found existing installation: wandb 0.12.18
    Uninstalling wandb-0.12.18:
      Successfully uninstalled wandb-0.12.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.3 requires wandb<0.13.0,>=0.10.0, but you have wandb 0.13.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pwd

'/kaggle/working'

In [5]:
cd ./SKD

/kaggle/working/SKD


In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 22.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 MB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 21.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 27.3 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 21.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.0/752.0 MB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 25.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 27.5 MB/s eta 0:00:0000:010:01m
  Preparing metadata (setup.py

In [7]:
!conda install mkl-service -y

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - mkl-service


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    mkl-service-2.4.0          |   py37hc4a68ce_0          61 KB  conda-forge
    openssl-1.1.1q             |       h166bdaf_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  mkl-service        conda-forge/linux-64::mkl-service-2.4.0-py37hc4a68ce_0

The following packages will be UPDATED:

  openssl                                 1.1.1o-h166bdaf_0 --> 1.1.1q-h166bdaf_0



openssl-1.1.1q       | 2.1 MB    | ##################################### | 100% 
mkl-service-2.4.0    | 61 KB     | ##################################### | 100% 
Preparing transaction: done
V

In [8]:
%%writefile /kaggle/working/SKD/models/__init__.py

from .resnet_ssl import resnet12_ssl,resnet50,resnet101,seresnet12,seresnet50,seresnet101


model_pool = [
    'resnet12_ssl',
    'resnet50',
    'resnet101',
    'seresnet12',
    'seresnet50',
    'seresnet101'
]

model_dict = {
    'resnet12_ssl': resnet12_ssl,
    'resnet50': resnet50,
    'resnet101': resnet101,
    'seresnet12': seresnet12,
    'seresnet50': seresnet50,
    'seresnet101':seresnet101
}


Overwriting /kaggle/working/SKD/models/__init__.py


In [9]:
%%writefile /kaggle/working/SKD/train_selfsupervison.py

from __future__ import print_function

import os
import argparse
import socket
import time
import sys
from tqdm import tqdm
import mkl

# import tensorboard_logger as tb_logger
import torch
import torch.optim as optim
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.autograd import Variable

from models import model_pool
from models.util import create_model

from dataset.mini_imagenet import ImageNet, MetaImageNet
from dataset.tiered_imagenet import TieredImageNet, MetaTieredImageNet
from dataset.cifar import CIFAR100, MetaCIFAR100
from dataset.transform_cfg import transforms_options, transforms_test_options, transforms_list

from util import adjust_learning_rate, accuracy, AverageMeter
from eval.meta_eval import meta_test, meta_test_tune
from eval.cls_eval import validate

from models.resnet import resnet12
import numpy as np
from util import Logger
import wandb
from dataloader import get_dataloaders

def get_freer_gpu():
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
    memory_available = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
    return np.argmax(memory_available)

os.environ["CUDA_VISIBLE_DEVICES"]=str(get_freer_gpu())
mkl.set_num_threads(2)


def parse_option():

    parser = argparse.ArgumentParser('argument for training')

    parser.add_argument('--eval_freq', type=int, default=10, help='meta-eval frequency')
    parser.add_argument('--print_freq', type=int, default=100, help='print frequency')
    parser.add_argument('--tb_freq', type=int, default=500, help='tb frequency')
    parser.add_argument('--save_freq', type=int, default=10, help='save frequency')
    parser.add_argument('--batch_size', type=int, default=64, help='batch_size')
    parser.add_argument('--num_workers', type=int, default=8, help='num of workers to use')
    parser.add_argument('--epochs', type=int, default=100, help='number of training epochs')

    # optimization
    parser.add_argument('--learning_rate', type=float, default=0.05, help='learning rate')
    parser.add_argument('--lr_decay_epochs', type=str, default='60,80', help='where to decay lr, can be a list')
    parser.add_argument('--lr_decay_rate', type=float, default=0.1, help='decay rate for learning rate')
    parser.add_argument('--weight_decay', type=float, default=5e-4, help='weight decay')
    parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
    parser.add_argument('--adam', action='store_true', help='use adam optimizer')
    parser.add_argument('--simclr', type=bool, default=False, help='use simple contrastive learning representation')
    parser.add_argument('--ssl', type=bool, default=True, help='use self supervised learning')
    parser.add_argument('--tags', type=str, default="gen0, ssl", help='add tags for the experiment')


    # dataset
    parser.add_argument('--model', type=str, default='resnet12', choices=model_pool)
    parser.add_argument('--dataset', type=str, default='miniImageNet', choices=['miniImageNet', 'tieredImageNet',
                                                                                'CIFAR-FS', 'FC100'])
    parser.add_argument('--transform', type=str, default='A', choices=transforms_list)
    parser.add_argument('--use_trainval', type=bool, help='use trainval set')

    # cosine annealing
    parser.add_argument('--cosine', action='store_true', help='using cosine annealing')

    # specify folder
    parser.add_argument('--model_path', type=str, default='save/', help='path to save model')
    parser.add_argument('--tb_path', type=str, default='tb/', help='path to tensorboard')
    parser.add_argument('--data_root', type=str, default='/raid/data/IncrementLearn/imagenet/Datasets/MiniImagenet/', help='path to data root')

    # meta setting
    parser.add_argument('--n_test_runs', type=int, default=600, metavar='N',
                        help='Number of test runs')
    parser.add_argument('--n_ways', type=int, default=5, metavar='N',
                        help='Number of classes for doing each classification run')
    parser.add_argument('--n_shots', type=int, default=1, metavar='N',
                        help='Number of shots in test')
    parser.add_argument('--n_queries', type=int, default=15, metavar='N',
                        help='Number of query in test')
    parser.add_argument('--n_aug_support_samples', default=5, type=int,
                        help='The number of augmented samples for each meta test sample')
    parser.add_argument('--test_batch_size', type=int, default=1, metavar='test_batch_size',
                        help='Size of test batch)')

    parser.add_argument('-t', '--trial', type=str, default='1', help='the experiment id')
    
    
    
    #hyper parameters
    parser.add_argument('--gamma', type=float, default=2, help='loss cofficient for ssl loss')
    
    opt = parser.parse_args()

    if opt.dataset == 'CIFAR-FS' or opt.dataset == 'FC100':
        opt.transform = 'D'

    if opt.use_trainval:
        opt.trial = opt.trial + '_trainval'

    # set the path according to the environment
    if not opt.model_path:
        opt.model_path = './models_pretrained'
    if not opt.tb_path:
        opt.tb_path = './tensorboard'
    if not opt.data_root:
        opt.data_root = './data/{}'.format(opt.dataset)
    else:
        opt.data_root = '{}/{}'.format(opt.data_root, opt.dataset)
    opt.data_aug = True

    iterations = opt.lr_decay_epochs.split(',')
    opt.lr_decay_epochs = list([])
    for it in iterations:
        opt.lr_decay_epochs.append(int(it))
        
    tags = opt.tags.split(',')
    opt.tags = list([])
    for it in tags:
        opt.tags.append(it)

    opt.model_name = '{}_{}_lr_{}_decay_{}_trans_{}'.format(opt.model, opt.dataset, opt.learning_rate,
                                                            opt.weight_decay, opt.transform)

    if opt.cosine:
        opt.model_name = '{}_cosine'.format(opt.model_name)

    if opt.adam:
        opt.model_name = '{}_useAdam'.format(opt.model_name)

    opt.model_name = '{}_trial_{}'.format(opt.model_name, opt.trial)

    opt.tb_folder = os.path.join(opt.tb_path, opt.model_name)
    if not os.path.isdir(opt.tb_folder):
        os.makedirs(opt.tb_folder)

    opt.save_folder = os.path.join(opt.model_path, opt.model_name)
    if not os.path.isdir(opt.save_folder):
        os.makedirs(opt.save_folder)

    opt.n_gpu = torch.cuda.device_count()
    
    
    #extras
    opt.fresh_start = True
    
    
    return opt


def main():

    opt = parse_option()
    wandb.init(project=opt.model_path.split("/")[-1], tags=opt.tags)
    wandb.config.update(opt)
    wandb.save('*.py')
    wandb.run.save()
    
        
    train_loader, val_loader, meta_testloader, meta_valloader, n_cls = get_dataloaders(opt)

    # model
    model = create_model(opt.model, n_cls, opt.dataset)
    wandb.watch(model)
    
    # optimizer
    if opt.adam:
        print("Adam")
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=opt.learning_rate,
                                     weight_decay=0.0005)
    else:
        print("SGD")
        optimizer = optim.SGD(model.parameters(),
                              lr=opt.learning_rate,
                              momentum=opt.momentum,
                              weight_decay=opt.weight_decay)
        
        

    criterion = nn.CrossEntropyLoss()

    if torch.cuda.is_available():
        if opt.n_gpu > 1:
            model = nn.DataParallel(model)
        model = model.cuda()
        criterion = criterion.cuda()
        cudnn.benchmark = True

    # set cosine annealing scheduler
    if opt.cosine:
        eta_min = opt.learning_rate * (opt.lr_decay_rate ** 3)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, opt.epochs, eta_min, -1)

    # routine: supervised pre-training
    for epoch in range(1, opt.epochs + 1):
            if opt.cosine:
                scheduler.step()
            else:
                adjust_learning_rate(epoch, opt, optimizer)
            print("==> training...")


            time1 = time.time()
            train_acc, train_acc5, train_loss = train(epoch, train_loader, model, criterion, optimizer, opt)
            time2 = time.time()
            print('epoch {}, total time {:.2f}'.format(epoch, time2 - time1))


            val_acc, val_acc_top5, val_loss = validate(val_loader, model, criterion, opt)


            #validate
            start = time.time()
            meta_val_acc, meta_val_std, meta_val_acc5, meta_val_std5 = meta_test(model, meta_valloader,use_logit=True)
            test_time = time.time() - start
            print('Meta Val Acc : {:.4f}, Meta Val std: {:.4f}, Time: {:.1f}'.format(meta_val_acc, meta_val_std, test_time))

            #evaluate
            start = time.time()
            meta_test_acc, meta_test_std, meta_test_acc5, meta_test_std5 = meta_test(model, meta_testloader,use_logit=True)
            test_time = time.time() - start
            print('Meta Test Acc: {:.4f}, Meta Test std: {:.4f}, Time: {:.1f}'.format(meta_test_acc, meta_test_std, test_time))


            # regular saving
            if epoch % opt.save_freq == 0 or epoch==opt.epochs:
                print('==> Saving...')
                state = {
                    'epoch': epoch,
                    'optimizer': optimizer.state_dict(),
                    'model': model.state_dict(),
                }            
                save_file = os.path.join(opt.save_folder, 'model_'+str(wandb.run.name)+'.pth')
                torch.save(state, save_file)

                #wandb saving
                torch.save(state, os.path.join(wandb.run.dir, "model.pth"))

                ## onnx saving
                #dummy_input = torch.autograd.Variable(torch.randn(1, 3, 32, 32)).cuda()
                #torch.onnx.export(model, dummy_input, os.path.join(wandb.run.dir, "model.onnx"))

            wandb.log({'epoch': epoch, 
                       'Train Acc': train_acc,
                       'Train Acc top 5': train_acc5,
                       'Train Loss': train_loss,
                       'Val Acc': val_acc,
                       'Val Acc top 5': val_acc_top5,
                       'Val Loss': val_loss,
                       'Meta Test Acc': meta_test_acc,
                       'Meta Test std': meta_test_std,
                       'Meta Val Acc': meta_val_acc,
                       'Meta Val std': meta_val_std,
                       'Meta Test Acc5': meta_test_acc5,
                       'Meta Test std5': meta_test_std5,
                       'Meta Val Acc5': meta_val_acc5,
                       'Meta Val std5': meta_val_std5
                      })

    #final report 
    generate_final_report(model, opt, wandb)
    
    #remove output.txt log file 
    output_log_file = os.path.join(wandb.run.dir, "output.log")
    if os.path.isfile(output_log_file):
        os.remove(output_log_file)
    else:    ## Show an error ##
        print("Error: %s file not found" % output_log_file)
        
        
        
def train(epoch, train_loader, model, criterion, optimizer, opt):
    """One epoch training"""
    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    end = time.time()
    with tqdm(train_loader, total=len(train_loader)) as pbar:
        for idx, (input, target, _) in enumerate(pbar):
            data_time.update(time.time() - end)

            input = input.float()
            if torch.cuda.is_available():
                input = input.cuda()
                target = target.cuda()
            
            
            batch_size = input.size()[0]
            x = input
            x_90 = x.transpose(2,3).flip(2)
            x_180 = x.flip(2).flip(3)
            x_270 = x.flip(2).transpose(2,3)
            generated_data = torch.cat((x, x_90, x_180, x_270),0)
            train_targets = target.repeat(4)
            
            rot_labels = torch.zeros(4*batch_size).cuda().long()
            for i in range(4*batch_size):
                if i < batch_size:
                    rot_labels[i] = 0
                elif i < 2*batch_size:
                    rot_labels[i] = 1
                elif i < 3*batch_size:
                    rot_labels[i] = 2
                else:
                    rot_labels[i] = 3

            # ===================forward=====================
            
            (_,_,_,_, feat), (train_logit, rot_logits) = model(generated_data, rot=True)
            
            rot_labels = F.one_hot(rot_labels.to(torch.int64), 4).float()
            loss_ss = torch.sum(F.binary_cross_entropy_with_logits(input = rot_logits, target = rot_labels))
            loss_ce = criterion(train_logit, train_targets)
            
            loss = opt.gamma * loss_ss + loss_ce
            
            acc1, acc5 = accuracy(train_logit, train_targets, topk=(1, 5))
            losses.update(loss.item(), input.size(0))
            top1.update(acc1[0], input.size(0))
            top5.update(acc5[0], input.size(0))

            # ===================backward=====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
          
            # ===================meters=====================
            batch_time.update(time.time() - end)
            end = time.time()
            
            
            pbar.set_postfix({"Acc@1":'{0:.2f}'.format(top1.avg.cpu().numpy()), 
                              "Acc@5":'{0:.2f}'.format(top5.avg.cpu().numpy(),2), 
                              "Loss" :'{0:.2f}'.format(losses.avg,2), 
                             })

    print('Train_Acc@1 {top1.avg:.3f} Train_Acc@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return top1.avg, top5.avg, losses.avg



def generate_final_report(model, opt, wandb):
    
    
    opt.n_shots = 1
    train_loader, val_loader, meta_testloader, meta_valloader, _ = get_dataloaders(opt)
    
    #validate
    meta_val_acc, meta_val_std, meta_val_acc5, meta_val_std5 = meta_test(model, meta_valloader, use_logit=True)
    
   # meta_val_acc_feat, meta_val_std_feat = meta_test(model, meta_valloader, use_logit=False)

    #evaluate
    meta_test_acc, meta_test_std, meta_test_acc5, meta_test_std5 = meta_test(model, meta_testloader, use_logit=True)
    
    #meta_test_acc_feat, meta_test_std_feat = meta_test(model, meta_testloader, use_logit=False)
        
    print('Meta Val Acc : {:.4f}, Meta Val std: {:.4f}'.format(meta_val_acc, meta_val_std))
    #print('Meta Val Acc (feat): {:.4f}, Meta Val std (feat): {:.4f}'.format(meta_val_acc_feat, meta_val_std_feat))
    print('Meta Test Acc: {:.4f}, Meta Test std: {:.4f}'.format(meta_test_acc, meta_test_std))
    #print('Meta Test Acc (feat): {:.4f}, Meta Test std (feat): {:.4f}'.format(meta_test_acc_feat, meta_test_std_feat))
    
    
    wandb.log({'Final Meta Test Acc @1': meta_test_acc,
               'Final Meta Test std @1': meta_test_std,
               'Final Meta Test Acc @5': meta_test_acc5,
               'Final Meta Test std @5': meta_test_std5,
               #'Final Meta Test Acc  (feat) @1': meta_test_acc_feat,
               #'Final Meta Test std  (feat) @1': meta_test_std_feat,
               'Final Meta Val Acc @1': meta_val_acc,
               'Final Meta Val std @1': meta_val_std,
               'Final Meta Val Acc @5': meta_val_acc5,
               'Final Meta Val std @5': meta_val_std5,
               #'Final Meta Val Acc   (feat) @1': meta_val_acc_feat,
               #'Final Meta Val std   (feat) @1': meta_val_std_feat
              })

    
   # opt.n_shots = 5
   # train_loader, val_loader, meta_testloader, meta_valloader, _ = get_dataloaders(opt)
    
    #validate
   # meta_val_acc, meta_val_std = meta_test(model, meta_valloader, use_logit=True)
    
    #meta_val_acc_feat, meta_val_std_feat = meta_test(model, meta_valloader, use_logit=False)

    #evaluate
   # meta_test_acc, meta_test_std = meta_test(model, meta_testloader, use_logit=True)
    
    #meta_test_acc_feat, meta_test_std_feat = meta_test(model, meta_testloader, use_logit=False)
        
   # print('Meta Val Acc : {:.4f}, Meta Val std: {:.4f}'.format(meta_val_acc, meta_val_std))
    #print('Meta Val Acc (feat): {:.4f}, Meta Val std (feat): {:.4f}'.format(meta_val_acc_feat, meta_val_std_feat))
   # print('Meta Test Acc: {:.4f}, Meta Test std: {:.4f}'.format(meta_test_acc, meta_test_std))
    #print('Meta Test Acc (feat): {:.4f}, Meta Test std (feat): {:.4f}'.format(meta_test_acc_feat, meta_test_std_feat))

    #wandb.log({'Final Meta Test Acc @5': meta_test_acc,
     #          'Final Meta Test std @5': meta_test_std,
               #'Final Meta Test Acc  (feat) @5': meta_test_acc_feat,
               #'Final Meta Test std  (feat) @5': meta_test_std_feat,
       #        'Final Meta Val Acc @5': meta_val_acc,
      #         'Final Meta Val std @5': meta_val_std,
               #'Final Meta Val Acc   (feat) @5': meta_val_acc_feat,
               #'Final Meta Val std   (feat) @5': meta_val_std_feat
        #      })
    
    
if __name__ == '__main__':
    main()


Overwriting /kaggle/working/SKD/train_selfsupervison.py


In [10]:
%%writefile /kaggle/working/SKD/eval/meta_eval.py 

from __future__ import print_function

import numpy as np
import scipy
from scipy.stats import t
from tqdm import tqdm

import torch
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn as nn
import sys, os
from collections import Counter


sys.path.append(os.path.abspath('..'))

from util import accuracy


def mean_confidence_interval(data, data5, confidence=0.95):
    a = 100.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * t._ppf((1+confidence)/2., n-1)
    
    a5 = 100.0 * np.array(data5)
    n5 = len(a5)
    m5, se5 = np.mean(a5), scipy.stats.sem(a5)
    h5 = se5 * t._ppf((1+confidence)/2., n5-1)
    return m, h, m5, h5


def normalize(x):
    norm = x.pow(2).sum(1, keepdim=True).pow(1. / 2)
    out = x.div(norm)
    return out


def meta_test(net, testloader, use_logit=False, is_norm=True, classifier='LR'):
    net = net.eval()
    acc = []
    acc5 = []

    with torch.no_grad():
        with tqdm(testloader, total=len(testloader)) as pbar:
            for idx, data in enumerate(pbar):
                support_xs, support_ys, support_xs5, support_ys5, query_xs, query_ys, query_xs5, query_ys5 = data

                support_xs = support_xs.cuda()
                support_xs5 = support_xs5.cuda()
                query_xs = query_xs.cuda()
                query_xs5 = query_xs5.cuda()
                batch_size, _, height, width, channel = support_xs.size()
                batch_size5, _, height5, width5, channel5 = support_xs5.size()
                support_xs = support_xs.view(-1, height, width, channel)
                support_xs5 = support_xs5.view(-1, height5, width5, channel5)
                query_xs = query_xs.view(-1, height, width, channel)
                query_xs5 = query_xs5.view(-1, height5, width5, channel5)

                
                
#                 batch_size = support_xs.size()[0]
#                 x = support_xs
#                 x_90 = x.transpose(2,3).flip(2)
#                 x_180 = x.flip(2).flip(3)
#                 x_270 = x.flip(2).transpose(2,3)
#                 generated_data = torch.cat((x, x_90, x_180, x_270),0)
#                 support_ys = support_ys.repeat(1,4)
#                 support_xs = generated_data
            
#                 print(support_xs.size())
#                 print(support_ys.size())



                if use_logit:
                    support_features = net(support_xs).view(support_xs.size(0), -1)
                    support_features5 = net(support_xs5).view(support_xs5.size(0), -1)
                    query_features = net(query_xs).view(query_xs.size(0), -1)
                    query_features5 = net(query_xs5).view(query_xs5.size(0), -1)
                else:
                    feat_support, _ = net(support_xs, is_feat=True)
                    support_features = feat_support[-1].view(support_xs.size(0), -1)
                    feat_query, _ = net(query_xs, is_feat=True)
                    query_features = feat_query[-1].view(query_xs.size(0), -1)

#                     feat_support, _ = net(support_xs)
#                     support_features = feat_support.view(support_xs.size(0), -1)
#                     feat_query, _ = net(query_xs)
#                     query_features = feat_query.view(query_xs.size(0), -1)


                if is_norm:
                    support_features = normalize(support_features)
                    support_features5 = normalize(support_features5)
                    query_features = normalize(query_features)
                    query_features5 = normalize(query_features5)

                support_features = support_features.detach().cpu().numpy()
                support_features5 = support_features5.detach().cpu().numpy()
                query_features = query_features.detach().cpu().numpy()
                query_features5 = query_features5.detach().cpu().numpy()
                
                support_ys = support_ys.view(-1).numpy()
                support_ys5 = support_ys5.view(-1).numpy()
                query_ys = query_ys.view(-1).numpy()
                query_ys5 = query_ys5.view(-1).numpy()
                
                
                
                if classifier == 'LR':
                    clf = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000, penalty='l2',
                                             multi_class='multinomial')
                    clf5 = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000, penalty='l2',
                                             multi_class='multinomial')                         
                    clf.fit(support_features, support_ys)
                    clf5.fit(support_features5, support_ys5)
                    query_ys_pred = clf.predict(query_features)
                    query_ys_pred5 = clf5.predict(query_features5)
                elif classifier == 'NN':
                    query_ys_pred = NN(support_features, support_ys, query_features)
                elif classifier == 'Cosine':
                    query_ys_pred = Cosine(support_features, support_ys, query_features)
                else:
                    raise NotImplementedError('classifier not supported: {}'.format(classifier))

                    
#                 bs = query_features.shape[0]//opt.n_aug_support_samples
#                 a = np.reshape(query_ys_pred[:bs], (-1,1))
#                 c = query_ys[:bs]
#                 for i in range(1,opt.n_aug_support_samples):
#                     a = np.hstack([a, np.reshape(query_ys_pred[i*bs:(i+1)*bs], (-1,1))])
                
#                 d = [] 
#                 for i in range(a.shape[0]):
#                     b = Counter(a[i,:])
#                     d.append(b.most_common(1)[0][0])
                
# #                 (values,counts) = np.unique(a,axis=1, return_counts=True)
# #                 print(counts)
# # ind=np.argmax(counts)
# # print values[ind]  # pr


# # #                 a = np.argmax
# #                 print(a.shape)
# #                 print(c.shape)
                    
                acc.append(metrics.accuracy_score(query_ys, query_ys_pred))
                acc5.append(metrics.accuracy_score(query_ys5, query_ys_pred5))
                
                pbar.set_postfix({"FSL_Acc":'{0:.2f}'.format(metrics.accuracy_score(query_ys, query_ys_pred))})
    
    return mean_confidence_interval(acc,acc5)




def meta_test_tune(net, testloader, use_logit=False, is_norm=True, classifier='LR', lamda=0.2):
    net = net.eval()
    acc = []
    
    with tqdm(testloader, total=len(testloader)) as pbar:
        for idx, data in enumerate(pbar):
            support_xs, support_ys, query_xs, query_ys, support_ts, query_ts = data

            support_xs = support_xs.cuda()
            support_ys = support_ys.cuda()
            query_ys = query_ys.cuda()
            query_xs = query_xs.cuda()
            batch_size, _, height, width, channel = support_xs.size()
            support_xs = support_xs.view(-1, height, width, channel)
            support_ys = support_ys.view(-1,1)
            query_ys = query_ys.view(-1)
            query_xs = query_xs.view(-1, height, width, channel)

            if use_logit:
                support_features = net(support_xs).view(support_xs.size(0), -1)
                query_features = net(query_xs).view(query_xs.size(0), -1)
            else:
                feat_support, _ = net(support_xs, is_feat=True)
                support_features = feat_support[-1].view(support_xs.size(0), -1)
                feat_query, _ = net(query_xs, is_feat=True)
                query_features = feat_query[-1].view(query_xs.size(0), -1)

            if is_norm:
                support_features = normalize(support_features)
                query_features = normalize(query_features)
               
            y_onehot = torch.FloatTensor(support_ys.size()[0], 5).cuda()

            # In your for loop
            y_onehot.zero_()
            y_onehot.scatter_(1, support_ys, 1)

    
            X = support_features
            XTX = torch.matmul(torch.t(X),X)
            
            B = torch.matmul( (XTX + lamda*torch.eye(640).cuda() ).inverse(), torch.matmul(torch.t(X), y_onehot.float()) )
#             print(B.size())
            m = nn.Sigmoid()
            Y_pred = m(torch.matmul(query_features, B))
                
                
#             print(Y_pred, query_ys)
#             model = nn.Sequential(nn.Linear(64, 10),nn.LogSoftmax(dim=1))
#             optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#             criterion = nn.CrossEntropyLoss()

#             model.cuda()
#             criterion.cuda()
#             model.train()
            
#             for i in range(5):
#                 output = model(support_features)
#                 loss = criterion(output, support_ys)
#                 optimizer.zero_grad()
#                 loss.backward(retain_graph=True) # auto-grad 
#                 optimizer.step() # update  weights 
            
#             model.eval()
#             query_ys_pred = model(query_features)

            acc1, acc5 = accuracy(Y_pred, query_ys, topk=(1, 1))
            
            
#             support_features = support_features.detach().cpu().numpy()
#             query_features = query_features.detach().cpu().numpy()

#             support_ys = support_ys.view(-1).numpy()
#             query_ys = query_ys.view(-1).numpy()

#             if classifier == 'LR':
#                 clf = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000,
#                                          multi_class='multinomial')
#                 clf.fit(support_features, support_ys)
#                 query_ys_pred = clf.predict(query_features)
#             elif classifier == 'NN':
#                 query_ys_pred = NN(support_features, support_ys, query_features)
#             elif classifier == 'Cosine':
#                 query_ys_pred = Cosine(support_features, support_ys, query_features)
#             else:
#                 raise NotImplementedError('classifier not supported: {}'.format(classifier))

            acc.append(acc1.item()/100.0)

            pbar.set_postfix({"FSL_Acc":'{0:.4f}'.format(np.mean(acc))})
                
                
    return mean_confidence_interval(acc)



def meta_test_ensamble(net, testloader, use_logit=True, is_norm=True, classifier='LR'):
    for n in net:
        n = n.eval()
    acc = []

    with torch.no_grad():
        with tqdm(testloader, total=len(testloader)) as pbar:
            for idx, data in enumerate(pbar):
                support_xs, support_ys, query_xs, query_ys = data

                support_xs = support_xs.cuda()
                query_xs = query_xs.cuda()
                batch_size, _, height, width, channel = support_xs.size()
                support_xs = support_xs.view(-1, height, width, channel)
                query_xs = query_xs.view(-1, height, width, channel)

                if use_logit:
                    support_features = net[0](support_xs).view(support_xs.size(0), -1)
                    query_features = net[0](query_xs).view(query_xs.size(0), -1)
                    for n in net[1:]:
                        support_features += n(support_xs).view(support_xs.size(0), -1)
                        query_features += n(query_xs).view(query_xs.size(0), -1)
                else:
                    feat_support, _ = net(support_xs, is_feat=True)
                    support_features = feat_support[-1].view(support_xs.size(0), -1)
                    feat_query, _ = net(query_xs, is_feat=True)
                    query_features = feat_query[-1].view(query_xs.size(0), -1)

                if is_norm:
                    support_features = normalize(support_features)
                    query_features = normalize(query_features)

                support_features = support_features.detach().cpu().numpy()
                query_features = query_features.detach().cpu().numpy()

                support_ys = support_ys.view(-1).numpy()
                query_ys = query_ys.view(-1).numpy()

                if classifier == 'LR':
                    clf = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000,
                                             multi_class='multinomial')
                    clf.fit(support_features, support_ys)
                    query_ys_pred = clf.predict(query_features)
                elif classifier == 'NN':
                    query_ys_pred = NN(support_features, support_ys, query_features)
                elif classifier == 'Cosine':
                    query_ys_pred = Cosine(support_features, support_ys, query_features)
                else:
                    raise NotImplementedError('classifier not supported: {}'.format(classifier))

                acc.append(metrics.accuracy_score(query_ys, query_ys_pred))
                
                pbar.set_postfix({"FSL_Acc":'{0:.2f}'.format(metrics.accuracy_score(query_ys, query_ys_pred))})
                
    return mean_confidence_interval(acc)


def NN(support, support_ys, query):
    """nearest classifier"""
    support = np.expand_dims(support.transpose(), 0)
    query = np.expand_dims(query, 2)

    diff = np.multiply(query - support, query - support)
    distance = diff.sum(1)
    min_idx = np.argmin(distance, axis=1)
    pred = [support_ys[idx] for idx in min_idx]
    return pred


def Cosine(support, support_ys, query):
    """Cosine classifier"""
    support_norm = np.linalg.norm(support, axis=1, keepdims=True)
    support = support / support_norm
    query_norm = np.linalg.norm(query, axis=1, keepdims=True)
    query = query / query_norm

    cosine_distance = query @ support.transpose()
    max_idx = np.argmax(cosine_distance, axis=1)
    pred = [support_ys[idx] for idx in max_idx]
    return pred

Overwriting /kaggle/working/SKD/eval/meta_eval.py


In [11]:
%%writefile /kaggle/working/SKD/dataset/cifar.py 

from __future__ import print_function

import os
import pickle
from PIL import Image
import numpy as np

import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset


class CIFAR100(Dataset):
    """support FC100 and CIFAR-FS"""
    def __init__(self, args, partition='train', pretrain=True, is_sample=False, k=4096,
                 transform=None):
        super(Dataset, self).__init__()
        self.data_root = args.data_root
        self.partition = partition
        self.data_aug = args.data_aug
        self.mean = [0.5071, 0.4867, 0.4408]
        self.std = [0.2675, 0.2565, 0.2761]
        self.normalize = transforms.Normalize(mean=self.mean, std=self.std)
        self.pretrain = pretrain
        self.simclr = args.simclr
        
        
        if transform is None:
            if self.partition == 'train' and self.data_aug:
                self.transform = transforms.Compose([
                    lambda x: Image.fromarray(x),
                    transforms.RandomCrop(32, padding=4),
                    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                    transforms.RandomHorizontalFlip(),
                    lambda x: np.asarray(x),
                    transforms.ToTensor(),
                    self.normalize
                ])
            else:
                self.transform = transforms.Compose([
                    lambda x: Image.fromarray(x),
                    transforms.ToTensor(),
                    self.normalize
                ])
        else:
            self.transform = transform

        if self.pretrain:
            self.file_pattern = '%s.pickle'
        else:
            self.file_pattern = '%s.pickle'
        self.data = {}

        with open(os.path.join(self.data_root, self.file_pattern % partition), 'rb') as f:
            data = pickle.load(f, encoding='latin1')
            self.imgs = data['data']
            labels = data['labels']
            # adjust sparse labels to labels from 0 to n.
            cur_class = 0
            label2label = {}
            for idx, label in enumerate(labels):
                if label not in label2label:
                    label2label[label] = cur_class
                    cur_class += 1
            new_labels = []
            for idx, label in enumerate(labels):
                new_labels.append(label2label[label])
            self.labels = new_labels
        
        
        # pre-process for contrastive sampling
        self.k = k
        self.is_sample = is_sample
        if self.is_sample:
            self.labels = np.asarray(self.labels)
            self.labels = self.labels - np.min(self.labels)
            num_classes = np.max(self.labels) + 1

            self.cls_positive = [[] for _ in range(num_classes)]
            for i in range(len(self.imgs)):
                self.cls_positive[self.labels[i]].append(i)

            self.cls_negative = [[] for _ in range(num_classes)]
            for i in range(num_classes):
                for j in range(num_classes):
                    if j == i:
                        continue
                    self.cls_negative[i].extend(self.cls_positive[j])

            self.cls_positive = [np.asarray(self.cls_positive[i]) for i in range(num_classes)]
            self.cls_negative = [np.asarray(self.cls_negative[i]) for i in range(num_classes)]
            self.cls_positive = np.asarray(self.cls_positive)
            self.cls_negative = np.asarray(self.cls_negative)

    def __getitem__(self, item):
        img = np.asarray(self.imgs[item]).astype('uint8')
        target = self.labels[item] - min(self.labels)
        
        if(self.simclr):
            img1 = self.transform(img)
            img2 = self.transform(img)
            return (img1, img2), target, item
        
        img = self.transform(img)
        if not self.is_sample:
            return img, target, item
        else:
            pos_idx = item
            replace = True if self.k > len(self.cls_negative[target]) else False
            neg_idx = np.random.choice(self.cls_negative[target], self.k, replace=replace)
            sample_idx = np.hstack((np.asarray([pos_idx]), neg_idx))
            return img, target, item, sample_idx

    def __len__(self):
        return len(self.labels)
    
    
    
    
class CIFAR100_toy(Dataset):
    """support FC100 and CIFAR-FS"""
    def __init__(self, args, partition='train', pretrain=True, is_sample=False, k=4096,
                 transform=None):
        super(Dataset, self).__init__()
        self.data_root = args.data_root
        self.partition = partition
        self.data_aug = args.data_aug
        self.mean = [0.5071, 0.4867, 0.4408]
        self.std = [0.2675, 0.2565, 0.2761]
        self.normalize = transforms.Normalize(mean=self.mean, std=self.std)
        self.pretrain = pretrain
        self.simclr = args.simclr
        
        
        if transform is None:
            if self.partition == 'train' and self.data_aug:
                self.transform = transforms.Compose([
                    lambda x: Image.fromarray(x),
                    transforms.RandomCrop(32, padding=4),
                    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                    transforms.RandomHorizontalFlip(),
                    lambda x: np.asarray(x),
                    transforms.ToTensor(),
                    self.normalize
                ])
            else:
                self.transform = transforms.Compose([
                    lambda x: Image.fromarray(x),
                    transforms.ToTensor(),
                    self.normalize
                ])
        else:
            self.transform = transform

        if self.pretrain:
            self.file_pattern = '%s.pickle'
        else:
            self.file_pattern = '%s.pickle'
        self.data = {}

        with open(os.path.join(self.data_root, self.file_pattern % partition), 'rb') as f:
            data = pickle.load(f, encoding='latin1')
            self.imgs = data['data']
            labels = data['labels']
            # adjust sparse labels to labels from 0 to n.
            cur_class = 0
            label2label = {}
            for idx, label in enumerate(labels):
                if label not in label2label:
                    label2label[label] = cur_class
                    cur_class += 1
            new_labels = []
            for idx, label in enumerate(labels):
                new_labels.append(label2label[label])
            self.labels = new_labels
        
        self.labels = np.array(self.labels)
        self.imgs = np.array(self.imgs)
        print(self.labels.shape)
        print(self.imgs.shape)
        
        loc = np.where(self.labels<5)[0]
        self.labels = self.labels[loc]
        self.imgs   = self.imgs[loc]
        
        
        self.k = k
        self.is_sample = is_sample

    def __getitem__(self, item):
        img = np.asarray(self.imgs[item]).astype('uint8')
        target = self.labels[item] - min(self.labels)
        
        if(self.simclr):
            img1 = self.transform(img)
            img2 = self.transform(img)
            return (img1, img2), target, item
        
        img = self.transform(img)
        if not self.is_sample:
            return img, target, item
        else:
            pos_idx = item
            replace = True if self.k > len(self.cls_negative[target]) else False
            neg_idx = np.random.choice(self.cls_negative[target], self.k, replace=replace)
            sample_idx = np.hstack((np.asarray([pos_idx]), neg_idx))
            return img, target, item, sample_idx

    def __len__(self):
        return len(self.labels)
    
    
    


class MetaCIFAR100(CIFAR100):

    def __init__(self, args, partition='train', train_transform=None, test_transform=None, fix_seed=True):
        super(MetaCIFAR100, self).__init__(args, partition, False)
        self.fix_seed = fix_seed
        self.n_ways = args.n_ways
        self.n_shots = args.n_shots
        self.n_queries = args.n_queries
        self.classes = list(self.data.keys())
        self.n_test_runs = args.n_test_runs
        self.n_aug_support_samples = args.n_aug_support_samples
        if train_transform is None:
            self.train_transform = transforms.Compose([
                lambda x: Image.fromarray(x),
                transforms.RandomCrop(32, padding=4),
                transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                transforms.RandomHorizontalFlip(),
                lambda x: np.asarray(x),
                transforms.ToTensor(),
                self.normalize
            ])
        else:
            self.train_transform = train_transform

        if test_transform is None:
            self.test_transform = transforms.Compose([
                lambda x: Image.fromarray(x),
                transforms.ToTensor(),
                self.normalize
            ])
        else:
            self.test_transform = test_transform

        self.data = {}
        for idx in range(self.imgs.shape[0]):
            if self.labels[idx] not in self.data:
                self.data[self.labels[idx]] = []
            self.data[self.labels[idx]].append(self.imgs[idx])
        self.classes = list(self.data.keys())

    def __getitem__(self, item):
        if self.fix_seed:
            np.random.seed(item)
        cls_sampled = np.random.choice(self.classes, self.n_ways, False)
        
        support_xs = []
        support_ys = []
        support_ts = []
        query_xs = []
        query_ys = []
        query_ts = []
        
        support_xs5 = []
        support_ys5 = []
        support_ts5 = []
        query_xs5 = []
        query_ys5 = []
        query_ts5 = []
        
        for idx, cls in enumerate(cls_sampled):
            imgs = np.asarray(self.data[cls]).astype('uint8')
            support_xs_ids_sampled = np.random.choice(range(imgs.shape[0]), self.n_shots, False)
            support_xs.append(imgs[support_xs_ids_sampled])
            support_ys.append([idx] * self.n_shots)
            support_ts.append([cls] * self.n_shots)
                 
            support_xs_ids_sampled5 = np.random.choice(range(imgs.shape[0]), 5, False)
            support_xs5.append(imgs[support_xs_ids_sampled5])
            #support_xs5.append(imgs[support_xs_ids_sampled])
            support_ys5.append([idx] * 5)
            support_ts5.append([cls] * 5)
            
            query_xs_ids = np.setxor1d(np.arange(imgs.shape[0]), support_xs_ids_sampled)
            query_xs_ids = np.random.choice(query_xs_ids, self.n_queries, False)
            query_xs.append(imgs[query_xs_ids])
            query_ys.append([idx] * query_xs_ids.shape[0])
            query_ts.append([cls] * query_xs_ids.shape[0])
            
            query_xs_ids5 = np.setxor1d(np.arange(imgs.shape[0]), support_xs_ids_sampled5)
            query_xs_ids5 = np.random.choice(query_xs_ids5, self.n_queries, False)
            query_xs5.append(imgs[query_xs_ids5])
            query_ys5.append([idx] * query_xs_ids5.shape[0])
            query_ts5.append([cls] * query_xs_ids5.shape[0])
            
        support_xs, support_ys,support_xs5, support_ys5, query_xs, query_ys, query_xs5, query_ys5 = np.array(support_xs), np.array(support_ys),np.array(support_xs5), np.array(support_ys5), np.array(
            query_xs), np.array(query_ys), np.array(query_xs5),np.array(query_ys5)
        
        support_ts,support_ts5, query_ts, query_ts5 = np.array(support_ts),np.array(support_ts5), np.array(query_ts),np.array(query_ts5)
        
        num_ways, n_queries_per_way, height, width, channel = query_xs.shape
        query_xs = query_xs.reshape((num_ways * n_queries_per_way, height, width, channel))
        query_ys = query_ys.reshape((num_ways * n_queries_per_way,))
        query_ts = query_ts.reshape((num_ways * n_queries_per_way,))
        
        num_ways5, n_queries_per_way5, height5, width5, channel5 = query_xs5.shape
        query_xs5 = query_xs5.reshape((num_ways5 * n_queries_per_way5, height5, width5, channel5))
        query_ys5 = query_ys5.reshape((num_ways5 * n_queries_per_way5,))
        query_ts5 = query_ts5.reshape((num_ways5 * n_queries_per_way5,))

        support_xs = support_xs.reshape((-1, height, width, channel))
        support_xs5 = support_xs5.reshape((-1, height5, width5, channel5))
        
        
        
        
        
        if self.n_aug_support_samples > 1:
            support_xs = np.tile(support_xs, (self.n_aug_support_samples, 1, 1, 1))
            support_ys = np.tile(support_ys.reshape((-1,)), (self.n_aug_support_samples))
            support_ts = np.tile(support_ts.reshape((-1,)), (self.n_aug_support_samples))
            
            support_xs5 = np.tile(support_xs5, (self.n_aug_support_samples, 1, 1, 1))
            support_ys5 = np.tile(support_ys5.reshape((-1,)), (self.n_aug_support_samples))
            support_ts5 = np.tile(support_ts5.reshape((-1,)), (self.n_aug_support_samples))
        
        
        support_xs = np.split(support_xs, support_xs.shape[0], axis=0)
        support_xs5 = np.split(support_xs5, support_xs5.shape[0], axis=0)
        
        
        
        query_xs = query_xs.reshape((-1, height, width, channel))
        query_xs5 = query_xs5.reshape((-1, height5, width5, channel5))
        
        if self.n_aug_support_samples > 1:
            query_xs = np.tile(query_xs, (self.n_aug_support_samples, 1, 1, 1))
            query_ys = np.tile(query_ys.reshape((-1,)), (self.n_aug_support_samples))
            query_ts = np.tile(query_ts.reshape((-1,)), (self.n_aug_support_samples))
            
            query_xs5 = np.tile(query_xs5, (self.n_aug_support_samples, 1, 1, 1))
            query_ys5 = np.tile(query_ys5.reshape((-1,)), (self.n_aug_support_samples))
            query_ts5 = np.tile(query_ts5.reshape((-1,)), (self.n_aug_support_samples))
            
        query_xs = np.split(query_xs, query_xs.shape[0], axis=0)
        query_xs5 = np.split(query_xs5, query_xs5.shape[0], axis=0)

        support_xs = torch.stack(list(map(lambda x: self.train_transform(x.squeeze()), support_xs)))
        support_xs5 = torch.stack(list(map(lambda x: self.train_transform(x.squeeze()), support_xs5)))
        query_xs = torch.stack(list(map(lambda x: self.test_transform(x.squeeze()), query_xs)))
        query_xs5 = torch.stack(list(map(lambda x: self.test_transform(x.squeeze()), query_xs5)))

        return support_xs, support_ys, support_xs5, support_ys5, query_xs, query_ys, query_xs5, query_ys5

    def __len__(self):
        return self.n_test_runs


if __name__ == '__main__':
    args = lambda x: None
    args.n_ways = 5
    args.n_shots = 1
    args.n_queries = 12
    # args.data_root = 'data'
    args.data_root = '/home/yonglong/Downloads/FC100'
    args.data_aug = True
    args.n_test_runs = 5
    args.n_aug_support_samples = 1
    imagenet = CIFAR100(args, 'train')
    print(len(imagenet))
    print(imagenet.__getitem__(500)[0].shape)

    metaimagenet = MetaCIFAR100(args, 'train')
    print(len(metaimagenet))
    print(metaimagenet.__getitem__(500)[0].size())
    print(metaimagenet.__getitem__(500)[1].shape)
    print(metaimagenet.__getitem__(500)[2].size())
    print(metaimagenet.__getitem__(500)[3].shape)

Overwriting /kaggle/working/SKD/dataset/cifar.py


In [ ]:
#!wandb off

In [12]:
#Resnet_12_ssl
!python /kaggle/working/SKD/train_selfsupervison.py --model resnet12_ssl --model_path save/FC100 --dataset FC100 --data_root /kaggle/input/few-shotcifar-100 --epochs 20 --lr_decay_epochs 8 --use_trainval True

wandb: Tracking run with wandb version 0.8.36
wandb: Wandb version 0.13.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Run data is saved locally in wandb/run-20220809_101051-11gsx335
wandb: Syncing run stellar-morning-24
wandb: ⭐️ View project at https://app.wandb.ai/geolep/FC100
wandb: 🚀 View run at https://app.wandb.ai/geolep/FC100/runs/11gsx335
wandb: Run `wandb off` to turn off syncing.

*********** resnet12_ssl
SGD
==> training...
  0%|                                                   | 0/750 [00:00<?, ?it/s]/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.
/pytorch/tor